In [1]:
from qiskit import *
from qiskit.tools.visualization import plot_histogram
import array as arr

def good_linapx(_l_x, _l_fx):
	fx = arr.array('d', [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7])
	c1 = 0
	c2 = 0
	for i in range(16):
		r1 = 0
		r2 = 0
		l_x = _l_x
		l_fx = _l_fx
		x = i
		f_x = int(fx[i])
		while x > 0:
			r1 ^= (x & 1) & (l_x & 1)
			x >>= 1
			l_x >>= 1
		while f_x > 0:
			r2 ^= (f_x & 1) & (l_fx & 1)
			f_x >>= 1
			l_fx >>= 1
		if r1 == r2:
			c1 += 1
		else:
			c2 += 1
	return (c1 - c2)

In [2]:
qr = QuantumRegister(8)
circuit = QuantumCircuit(qr)

count_trivial = 0
count_bad_linapx = 0

circuit.h(qr[0])
circuit.h(qr[1])
circuit.h(qr[2])
circuit.h(qr[3])

circuit.x(qr[4])
circuit.rcccx(qr[0], qr[1], qr[2], qr[4])
circuit.rccx(qr[0], qr[1], qr[4])
circuit.cx(qr[0], qr[4])
circuit.rcccx(qr[1], qr[2], qr[3], qr[4])
circuit.rccx(qr[1], qr[2], qr[4])
circuit.cx(qr[1], qr[4])
circuit.cx(qr[3], qr[4])

circuit.x(qr[5])
circuit.rcccx(qr[0], qr[2], qr[3], qr[5])
circuit.rccx(qr[0], qr[2], qr[5])
circuit.cx(qr[0], qr[5])
circuit.rccx(qr[1], qr[3], qr[5])
circuit.cx(qr[1], qr[5])
circuit.rccx(qr[2], qr[3], qr[5])

circuit.x(qr[6])
circuit.rcccx(qr[0], qr[1], qr[2], qr[6])
circuit.rcccx(qr[0], qr[1], qr[3], qr[6])
circuit.rccx(qr[0], qr[1], qr[6])
circuit.rccx(qr[0], qr[2], qr[6])
circuit.rccx(qr[0], qr[3], qr[6])
circuit.rccx(qr[1], qr[2], qr[6])
circuit.rccx(qr[1], qr[3], qr[6])
circuit.rccx(qr[2], qr[3], qr[6])
circuit.cx(qr[2], qr[6])
circuit.cx(qr[3], qr[6])

circuit.rcccx(qr[0], qr[2], qr[3], qr[7])
circuit.rccx(qr[0], qr[3], qr[7])
circuit.cx(qr[0], qr[7])
circuit.rccx(qr[1], qr[3], qr[7])
circuit.cx(qr[2], qr[7])

circuit.h(qr[0])
circuit.h(qr[1])
circuit.h(qr[2])
circuit.h(qr[3])
circuit.h(qr[4])
circuit.h(qr[5])
circuit.h(qr[6])
circuit.h(qr[7])

circuit.measure_all()

In [3]:
from qiskit import IBMQ
IBMQ.save_account('')
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')
qcomp = provider.get_backend('ibmq_16_melbourne')

In [7]:
job = execute(circuit, backend = qcomp)
from qiskit.tools.monitor import job_monitor
job_monitor(job)
counts = job.result().get_counts()

Job Status: job has successfully run


In [6]:
try:
	count_trivial = counts['{0:08b}'.format(0)]
except:
	print('Trivial solution occurred 0 time')
if count_trivial != 0:
	print('Trivial solution occur ' + str(count_trivial) + ' times')

count_bad_linapx = 0
for i in range(256):
	if '{0:08b}'.format(i) in counts:
		occurs = counts['{0:08b}'.format(i)]
		if occurs > 0 and good_linapx((i >> 4), (i & 15)) == 0:
			count_bad_linapx += 1

print('Bad linear approximation occur ' + str(count_bad_linapx) + ' times')
print(counts)

Trivial solution occur 8 times
Bad linear approximation occur 109 times
{'00100011': 1, '10000101': 9, '11111000': 3, '00101111': 1, '00100100': 3, '00001011': 4, '01100111': 2, '11011000': 5, '11111001': 5, '10011011': 9, '10100010': 3, '10000001': 9, '00111001': 5, '11001110': 2, '00101101': 5, '00101110': 4, '00111011': 3, '01110011': 4, '10110111': 3, '00110000': 4, '10011100': 7, '10010011': 3, '10100011': 2, '11001010': 5, '00011101': 10, '01000011': 5, '10011001': 10, '11010000': 6, '00001111': 9, '11100111': 1, '00000001': 17, '01101100': 1, '01011100': 3, '11011001': 6, '11011111': 2, '11101011': 2, '01111110': 1, '11111011': 2, '00101010': 2, '11011110': 2, '10111100': 1, '11100110': 1, '01010011': 6, '10010001': 6, '01011101': 8, '00110101': 2, '11101111': 2, '11001001': 10, '10110011': 1, '10000100': 7, '10011110': 4, '01110101': 2, '00111101': 1, '10011101': 2, '11111100': 1, '01111010': 1, '11000001': 4, '00010100': 8, '11101101': 1, '01110010': 2, '11001000': 4, '0000000